## Run bactrack 
- Computer segementation Hierarchies for all frames
- Run tracking on Hierarchies
  - Choose a weight function for linking
  - Choose a solver for mip solving
- Output/store the result

1. Computer segementation Hierarchies by choosing hypermodel, and submodel 

In [ ]:
from bactrack.core import compute_hierarchy
dir = "/Users/sherryyang/Documents/wiggins-lab/data/data_master/data6"
# chose hypermodel between Cellpose and Omnipose
# chose submodel under instruction of hypermodel
hier_arr = compute_hierarchy(dir,hypermodel='omnipose', submodel= 'bact_phase_omni')

2. Run tracking 
  - choose a weight function you would like to use

In [ ]:
from bactrack.tracking import OverlapWeight, IOUWeight, DistanceWeight
w =  DistanceWeight(hier_arr)

- Choose a MIP Solver you would like to use
   - MIPSolver = python_mip repo: CBC, GUROBI
   - ScipySolver = scipy.optimize.milp: HiGHS 

In [ ]:
#   MIPSolver => python_mip repo: it includes two solvers: CBC and Gurobi
#   ScipySolver => scipy.optimize.milp: it include HiGHS solvers

# MIPSolver require conda environment and additional setup, so it not always avilable, you could use 
# ScipySolver instead
from bactrack.tracking import MIPSolver, ScipySolver

In [ ]:
# do solver = ScipySolver(w.weight_matrix, hier_arr, mask_penalty= w.mask_penalty)
# if you want include a mask_penalty calculated by omnipose/cellpose mask threshold + flow threshold
solver = ScipySolver(w.weight_matrix, hier_arr)

# n is a list of all selected segementation
# e is a scipy sparse matrix mark the edge be selected
n, e = solver.solve()

Instead of choose weight and solver manually, you could also do following, this give you same result as solver.solve()

In [ ]:
from bactrack import run_tracking
n, e = run_tracking(hier_arr, weight_name='overlap_weight', solver_name = "scipy_solver")

3.  Output/store the result 
- Output result in dataframes, and make selected segementation result into a mask array

In [ ]:
from bactrack import io
# m is a mask array including all output masks
# e is a edge dataframe
# also selected cell in  hier_arr be marked in this function
m, e = io.format_output(hier_arr, n, e)
# now n is  a node dataframe
n = io.hiers_to_df(hier_arr)

- Store resuslt
  - For dataframes: feel free to call any dataframe store function to store the data in the format you want, to_pickle is good to maintain the data type. 
  - For mask array do store_mask


In [ ]:
root_dir = "./"
n.to_pickle(root_dir + "nodes.pkl")
e.to_pickle(root_dir + "dges.pkl")
io.store_mask_arr(m, root_dir+"masks")